## Deriving Tropospheric NO₂ Concentration Using Google Earth Engine from HUCs

This notebook documents the methodology used to extract monthly tropospheric NO₂ concentrations over Philippine cities using Sentinel-5P satellite data via Google Earth Engine (GEE). Same extraction is done for extracting monthy tropospheric NO2 concentration over target Philippine Regions

### 1. Region Definition: Philippine Cities and Municipalities

We began by loading the Level-2 administrative boundaries (ADM3) for the Philippines:


In [ ]:
var level2 = ee.FeatureCollection("projects/air-quality-monitoring-461610/assets/PH_Adm3_MuniCities");
print("Sample Level-2 units:", level2.limit(5));

From this, we selected 33 Highly Urbanized Cities (HUCs) by filtering `adm3_en`:

In [ ]:
var hucNames = [
    "Quezon City", "City of Davao", "City of Makati", "City of Cebu", "City of Baguio",
    "City of Iloilo", "City of Zamboanga", "City of Pasig", "City of Mandaluyong",
    "City of Muntinlupa", "City of Parañaque", "City of Valenzuela", "City of Manila",
    "City of Las Piñas", "City of Caloocan", "City of Marikina", "City of Malabon",
    "City of Navotas", "City of Taguig", "Pasay City", "City of Tacloban", "City of Butuan",
    "City of Bacolod", "City of Angeles", "City of Lucena", "City of Olongapo",
    "City of General Santos", "City of Puerto Princesa", "City of Lapu-Lapu",
    "City of Mandaue", "Ormoc City", "City of Cagayan De Oro", "City of Iligan",
    "City of Cotabato", "Pateros", "City of Isabela", "City of San Juan"
  ];

var hucFilter = ee.Filter.inList('adm3_en', hucNames);
var hucFeatures = level2.filter(hucFilter);

These cities served as zones for aggregating NO₂ values.

### 2. Temporal Coverage: January 2018 – December 2025

The analysis period covers:

- Start: January 1, 2018  
- End: December 31, 2025

In [ ]:
var startDate = ee.Date('2018-01-01');
var endDate   = ee.Date('2026-01-01'); // Exclusive end date

We then generated monthly indices within this period:

In [ ]:
var nMonths = endDate.difference(startDate, 'month').toInt();
var monthIndices = ee.List.sequence(0, nMonths.subtract(1));

### 🛰 3. Satellite Data: Sentinel-5P OFFL NO₂ Product

We used the Sentinel-5P tropospheric NO₂ dataset:

- Dataset ID: `COPERNICUS/S5P/OFFL/L3_NO2`  
- Band: `NO2_column_number_density` (mol/m²)



### 🧮 4. Monthly Mean NO₂ Extraction

For each month in the date range:
- Filter satellite images to that month
- Compute the monthly mean image
- Apply zonal mean reduction over each HUC

In [ ]:
var perMonthCollections = monthIndices.map(function(monthOffset) {
var thisMonthStart = startDate.advance(monthOffset, 'month');
var nextMonthStart = thisMonthStart.advance(1, 'month');

var monthlyImage = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')
    .filterDate(thisMonthStart, nextMonthStart)
    .select('NO2_column_number_density')
    .mean();

var bandExists = monthlyImage.bandNames().contains('NO2_column_number_density');

return ee.Algorithms.If(
    bandExists,
    monthlyImage.reduceRegions({
    collection: hucFeatures,
    reducer: ee.Reducer.mean().setOutputs(['NO2']),
    scale: 1000,
    tileScale: 4
    }).map(function(cityFeature) {
    return cityFeature.set('month', thisMonthStart.format('YYYY-MM'));
    }),
    null
);
});

We removed `null` entries and flattened the result:

In [ ]:
var filteredList = ee.List(perMonthCollections).removeAll([null]);
var NO2_byHUC_Month = ee.FeatureCollection(filteredList).flatten();


### 📅 5. Temporal Tagging

Each feature was annotated with:

- `month`: formatted string (e.g., `"2021-07"`)
- `date`: actual date object (`YYYY-MM-01`)


In [ ]:
var NO2_withDate = NO2_byHUC_Month.map(function(f) {
    var monthString = ee.String(f.get('month'));
    var fullDateString = monthString.cat('-01');
    var dt = ee.Date(fullDateString);
    return f.set('date', dt);
});

### 📤 6. Exporting the Final Dataset

We exported the complete NO₂

In [ ]:
Export.table.toDrive({
    collection: NO2_withDate,
    description: 'PHL_HUCs_Monthly_NO2_2018_2025',
    fileFormat: 'CSV',
    selectors: ['adm3_en', 'month', 'NO2', 'date']
});


The CSV includes:
- `adm3_en`: city name
- `month`: month in `"YYYY-MM"` format
- `NO2`: mean concentration (mol/m²)
- `date`: full date object

The CSV file is found at `../datasets/no2_data`